In [133]:
## reset specific variables (replace regular_expression by the variables of interest)
#%reset_selective <regular_expression>

# reset all variables
%reset -f

In [1]:
## Importing libraries

from datetime import datetime, date, timedelta
from IPython.display import display, clear_output
import time
from array import *
import numpy as np
import gc
import os
from pathlib import Path
from pylab import savefig
import pandas as pd
import csv
import json
import math
import datetime as dt
import pymongo as pym
import statsmodels

In [2]:
## Variables to connect to the DB

myclient = pym.MongoClient("mongodb://localhost:27017/")
mydb = myclient["SMARTAttFullBackBlaze"]
mycol = mydb["s_m_a_r_t_att_full_back_blaze"]

# Selecting HDD model and attributes
#models = ["HGST HMS5C4040BLE640", "HGST HUH728080ALE600", "HGST HUH721212ALN604", "ST4000DM000", "ST8000NM0055", "ST12000NM0008", "ST12000NM0007", "ST14000NM001G", "ST14000NM0138", "ST16000NM001G", "TOSHIBA MD04ABA400V", "TOSHIBA MG07ACA14TA", "WDC WUH721414ALE6L4"]
model = "TOSHIBA MG07ACA14TA"
myquery = { "model": { "$eq": model }}
myfields = {"timestamp":1, "disk_id":1, "model":1, "failure":1, "_id":0, "r_sectors":1, "u_errors":1, "n_b_written":1, "command_timeout":1, "current_pending_sector_count":1, "uncorrectable_sector_count":1}

#Consulting
mydoc = mycol.find(myquery, myfields)

# Converting into DataFrame
list = list(mydoc)
dfAllInformationPerHDDModel =  pd.DataFrame(list)

In [3]:
## Printing dataframe last records to check that the DB has been read correctly

dfAllInformationPerHDDModel.tail()

,timestamp,disk_id,model,failure,r_sectors,u_errors,n_b_written,command_timeout,current_pending_sector_count,uncorrectable_sector_count
23288790,2022-06-26,Z9R0A08DF97G,TOSHIBA MG07ACA14TA,0.0,0.0,NaN,NaN,NaN,0.0,0.0
23288791,2022-06-27,Z9R0A08DF97G,TOSHIBA MG07ACA14TA,0.0,0.0,NaN,NaN,NaN,0.0,0.0
23288792,2022-06-28,Z9R0A08DF97G,TOSHIBA MG07ACA14TA,0.0,0.0,NaN,NaN,NaN,0.0,0.0
23288793,2022-06-29,Z9R0A08DF97G,TOSHIBA MG07ACA14TA,0.0,0.0,NaN,NaN,NaN,0.0,0.0
23288794,2022-06-30,Z9R0A08DF97G,TOSHIBA MG07ACA14TA,0.0,0.0,NaN,NaN,NaN,0.0,0.0


In [4]:
## Looking for those that didn't fail

operationalHDD = dfAllInformationPerHDDModel.loc[dfAllInformationPerHDDModel.failure.isin([0])]
operationalHDD

,timestamp,disk_id,model,failure,r_sectors,u_errors,n_b_written,command_timeout,current_pending_sector_count,uncorrectable_sector_count
0,2020-11-10,1030A001F97G,TOSHIBA MG07ACA14TA,0.0,0.0,NaN,NaN,NaN,0.0,0.0
1,2020-11-11,1030A001F97G,TOSHIBA MG07ACA14TA,0.0,0.0,NaN,NaN,NaN,0.0,0.0
2,2020-11-12,1030A001F97G,TOSHIBA MG07ACA14TA,0.0,0.0,NaN,NaN,NaN,0.0,0.0
3,2020-11-13,1030A001F97G,TOSHIBA MG07ACA14TA,0.0,0.0,NaN,NaN,NaN,0.0,0.0
4,2020-11-14,1030A001F97G,TOSHIBA MG07ACA14TA,0.0,0.0,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
23288790,2022-06-26,Z9R0A08DF97G,TOSHIBA MG07ACA14TA,0.0,0.0,NaN,NaN,NaN,0.0,0.0
23288791,2022-06-27,Z9R0A08DF97G,TOSHIBA MG07ACA14TA,0.0,0.0,NaN,NaN,NaN,0.0,0.0
23288792,2022-06-28,Z9R0A08DF97G,TOSHIBA MG07ACA14TA,0.0,0.0,NaN,NaN,NaN,0.0,0.0
23288793,2022-06-29,Z9R0A08DF97G,TOSHIBA MG07ACA14TA,0.0,0.0,NaN,NaN,NaN,0.0,0.0


In [5]:
## Looking for those that have failed

failedHDD = dfAllInformationPerHDDModel.loc[dfAllInformationPerHDDModel.failure.isin([1])]
failedHDD

,timestamp,disk_id,model,failure,r_sectors,u_errors,n_b_written,command_timeout,current_pending_sector_count,uncorrectable_sector_count
7137,2022-06-01,1030A051F97G,TOSHIBA MG07ACA14TA,1.0,0.0,NaN,NaN,NaN,0.0,1.0
12168,2022-05-11,1040A003F97G,TOSHIBA MG07ACA14TA,1.0,0.0,NaN,NaN,NaN,0.0,0.0
456488,2021-10-28,1090A03MF97G,TOSHIBA MG07ACA14TA,1.0,0.0,NaN,NaN,NaN,1.0,4.0
498782,2021-12-17,10A0A034F97G,TOSHIBA MG07ACA14TA,1.0,0.0,NaN,NaN,NaN,0.0,0.0
521201,2020-03-26,10A0A042F97G,TOSHIBA MG07ACA14TA,1.0,0.0,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
22956262,2020-01-17,Z8C0A00AF97G,TOSHIBA MG07ACA14TA,1.0,0.0,NaN,NaN,NaN,1.0,0.0
22971186,2022-05-19,Z8D0A00UF97G,TOSHIBA MG07ACA14TA,1.0,1.0,NaN,NaN,NaN,1.0,0.0
23038856,2020-11-29,Z9Q0A01GF97G,TOSHIBA MG07ACA14TA,1.0,2.0,NaN,NaN,NaN,0.0,0.0
23093863,2020-07-11,Z9Q0A040F97G,TOSHIBA MG07ACA14TA,1.0,0.0,NaN,NaN,NaN,0.0,0.0


In [6]:
## Saving data per storage model into a csv: allinformation, failures, operational

operationalHDD.to_csv('SplittedDatasetHDD/' + 'OperationalHDDPerModel' + model + '.csv')
failedHDD.to_csv('SplittedDatasetHDD/' + 'FailedHDDPerModel' + model + '.csv')
dfAllInformationPerHDDModel.to_csv('SplittedDatasetHDD/' + 'allInformationPerHDDModel' + model + '.csv')

In [32]:
## Loading csv per storage model

#models = ["HGST HMS5C4040BLE640", "HGST HUH728080ALE600", "HGST HUH721212ALN604", "ST4000DM000", "ST8000NM0055", "ST12000NM0008", "ST12000NM0007", "ST14000NM001G", "ST14000NM0138", "ST16000NM001G", "TOSHIBA MD04ABA400V", "TOSHIBA MG07ACA14TA", "WDC WUH721414ALE6L4"]

model = "WDC WUH721414ALE6L4"

dfAllInformationPerHDDModel = pd.read_csv('SplittedDatasetHDD/' + 'allInformationPerHDDModel' + model + '.csv')
failedHDD = pd.read_csv('SplittedDatasetHDD/' + 'FailedHDDPerModel' + model + '.csv')

In [33]:
## Extraction of metrics

#time when the first one was operational
timeFirstOperational = dfAllInformationPerHDDModel.timestamp.min()
#timeFirstOperational = datetime.strptime(timeFirstOperational, '%Y-%m-%d') #for when the data comes from a loaded csv

# time when the last one was monitored
timeLastOneOperational = dfAllInformationPerHDDModel.timestamp.max()
#timeLastOneOperational = datetime.strptime(timeLastOneOperational, '%Y-%m-%d') #for when the data comes from a loaded csv

# calculating period
rangeTime = timeLastOneOperational - timeFirstOperational

# total of different disks
totalDiffDisks = dfAllInformationPerHDDModel.disk_id.unique()
totalDiffDisks = len(totalDiffDisks)

# total of failed disks
totalFailedDisks = failedHDD.disk_id.count()

# total of operational disks (after failures)
totalOperationalDisks = totalDiffDisks - totalFailedDisks

# percentage of not null values for the following attributes
r_sectors_NoNull = ((dfAllInformationPerHDDModel.r_sectors.count())/(dfAllInformationPerHDDModel.model.count()))*100
u_errors_NoNull = ((dfAllInformationPerHDDModel.u_errors.count())/(dfAllInformationPerHDDModel.model.count()))*100
n_b_written_NoNull = ((dfAllInformationPerHDDModel.n_b_written.count())/(dfAllInformationPerHDDModel.model.count()))*100
command_timeout_NoNull = ((dfAllInformationPerHDDModel.command_timeout.count())/(dfAllInformationPerHDDModel.model.count()))*100
current_pending_sector_count_NoNull = ((dfAllInformationPerHDDModel.current_pending_sector_count.count())/(dfAllInformationPerHDDModel.model.count()))*100
uncorrectable_sector_count_NoNull = ((dfAllInformationPerHDDModel.uncorrectable_sector_count.count())/(dfAllInformationPerHDDModel.model.count()))*100

# total written blocks (ex: written blocks)
totalr_sectors = dfAllInformationPerHDDModel.r_sectors.sum()
totalu_errors = dfAllInformationPerHDDModel.u_errors.sum()
totaln_b_written = dfAllInformationPerHDDModel.n_b_written.sum()
totalcommand_timeout = dfAllInformationPerHDDModel.command_timeout.sum()
totalcurrent_pending_sector_count = dfAllInformationPerHDDModel.current_pending_sector_count.sum()
totaluncorrectable_sector_count = dfAllInformationPerHDDModel.uncorrectable_sector_count.sum()

In [34]:
# Storing the metrics into a dataframe
arrayExtractedMetrics = np.array([[timeFirstOperational, timeLastOneOperational, rangeTime, totalDiffDisks, totalFailedDisks, totalOperationalDisks, r_sectors_NoNull, u_errors_NoNull, n_b_written_NoNull, command_timeout_NoNull, current_pending_sector_count_NoNull, uncorrectable_sector_count_NoNull, totalr_sectors, totalu_errors, totaln_b_written, totalcommand_timeout, totalcurrent_pending_sector_count, totaluncorrectable_sector_count]])
dfExtractedMetrics = pd.DataFrame(arrayExtractedMetrics, columns=['timeFirstOperational','timeLastOneOperational','rangeTime','totalDiffDisks','totalFailedDisks','totalOperationalDisks','r_sectors_NoNull','u_errors_NoNull','n_b_written_NoNull','command_timeout_NoNull','current_pending_sector_count_NoNull','uncorrectable_sector_count_NoNull','totalr_sectors','totalu_errors','totaln_b_written','totalcommand_timeout','totalcurrent_pending_sector_count','totaluncorrectable_sector_count'])
dfExtractedMetrics.to_csv('SplittedDatasetHDD/' + 'extractedMetrics' + model + '.csv')

In [35]:
dfExtractedMetrics.head()

,timeFirstOperational,timeLastOneOperational,rangeTime,totalDiffDisks,totalFailedDisks,totalOperationalDisks,r_sectors_NoNull,u_errors_NoNull,n_b_written_NoNull,command_timeout_NoNull,current_pending_sector_count_NoNull,uncorrectable_sector_count_NoNull,totalr_sectors,totalu_errors,totaln_b_written,totalcommand_timeout,totalcurrent_pending_sector_count,totaluncorrectable_sector_count
0,2020-10-07,2022-06-30,631 days,8446,37,8409,99.89677,0.0,0.0,0.0,99.89677,99.89677,221746.0,0.0,0.0,0.0,346856.0,14561.0


In [33]:
# Example of aggregation to extract some metrics

tempDataGeneral = dfAllInformationPerHDDModel.groupby(['timestamp', 'model'], as_index=False).sum()
tempDataGeneral

In [ ]:
tempDataSpfParam = dfAllInformationPerHDDModel.groupby(['timestamp', 'model'], as_index=False)['n_b_written'].sum()